In [3]:
import os
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
# 1. PATHS (Based on your upload)
real_path = r"C:\Users\hajar\Downloads\ML_Projrct\ML_Projrct\VoiceGuard_samples\Real\R.wav"
fake_path = r"C:\Users\hajar\Downloads\ML_Projrct\ML_Projrct\VoiceGuard_samples\Fake\F.mp3"
model_path = r"C:\Users\hajar\Downloads\ML_Projrct\ML_Projrct\best_large_model.keras" "

# 2. SPECTROGRAM SETTINGS (Must match training EXACTLY)
N_MELS = 128
FIXED_WIDTH = 300

# --- HELPER FUNCTIONS ---
def prepare_data(file_path):
    # Load
    y, sr = librosa.load(file_path, sr=16000)
    
    # Spectrogram
    mel_spec = librosa.feature.melspectrogram(
        y=y, sr=sr, n_mels=N_MELS, n_fft=2048, hop_length=160
    )
    mel_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Pad/Crop to FIXED_WIDTH
    if mel_db.shape[1] < FIXED_WIDTH:
        pad_width = FIXED_WIDTH - mel_db.shape[1]
        mel_db = np.pad(mel_db, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mel_db = mel_db[:, :FIXED_WIDTH]
        
    # Prepare for Model (Reshape & Normalize)
    model_input = mel_db.reshape(1, N_MELS, FIXED_WIDTH, 1)
    model_input = (model_input - (-40)) / 20 # Simple normalization
    
    return mel_db, model_input

# --- MAIN EXECUTION ---

# 1. Load Model
if os.path.exists(model_path):
    print("🧠 Loading Model...")
    model = tf.keras.models.load_model(model_path)
else:
    print("❌ Error: Model file not found. Please output/save your model first.")
    model = None

# 2. Process Files
files = [("Real File", real_path), ("Fake XTTS File", fake_path)]

plt.figure(figsize=(15, 12))

for i, (title, path) in enumerate(files):
    if not os.path.exists(path):
        print(f"⚠️ File not found: {path}")
        continue
        
    # Get Data
    vis_data, model_data = prepare_data(path)
    
    # Predict
    prediction_text = "Model not loaded"
    color = "black"
    if model:
        pred = model.predict(model_data, verbose=0)[0][0]
        confidence = pred * 100
        
        if pred > 0.5:
            prediction_text = f"DETECTED: FAKE ({confidence:.2f}% Confidence)"
            color = "red"
        else:
            prediction_text = f"DETECTED: REAL ({100-confidence:.2f}% Confidence)"
            color = "green"

    # Plot Spectrogram
    plt.subplot(2, 1, i+1)
    librosa.display.specshow(vis_data, sr=16000, hop_length=160, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"{title}\nAI Verdict: {prediction_text}", fontsize=14, color=color, fontweight='bold')

plt.tight_layout()
plt.show()

SyntaxError: unterminated string literal (detected at line 12) (1755959580.py, line 12)

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tempfile
import os

# Page Config
st.set_page_config(page_title="Deepfake Voice Detector", page_icon="🎙")

# Load Model
@st.cache_resource
def load_my_model():
    if not os.path.exists("best_large_model.keras"):
        st.error("Model not found! Make sure 'best_large_model.keras' is in the output.")
        return None
    return tf.keras.models.load_model("best_large_model.keras")

model = load_my_model()

# Preprocessing (Must match training)
def prepare_audio(file_path):
    N_MELS = 128
    FIXED_WIDTH = 300
    try:
        y, sr = librosa.load(file_path, sr=16000)
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS, n_fft=2048, hop_length=160)
        mel_db = librosa.power_to_db(mel_spec, ref=np.max)
        
        if mel_db.shape[1] < FIXED_WIDTH:
            pad_width = FIXED_WIDTH - mel_db.shape[1]
            mel_db = np.pad(mel_db, ((0, 0), (0, pad_width)), mode='constant')
        else:
            mel_db = mel_db[:, :FIXED_WIDTH]
            
        model_input = mel_db.reshape(1, N_MELS, FIXED_WIDTH, 1)
        model_input = (model_input - (-40)) / 20 
        return mel_db, model_input
    except Exception as e:
        return None, None

# UI Layout
st.title("🎙 AI Deepfake Detector")
uploaded_file = st.file_uploader("Upload Audio (wav/mp3)", type=["wav", "mp3"])

if uploaded_file and model:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
        tmp.write(uploaded_file.read())
        path = tmp.name
    
    st.audio(path)
    
    if st.button("Analyze"):
        vis, inp = prepare_audio(path)
        if inp is not None:
            prob = model.predict(inp, verbose=0)[0][0]
            conf = prob * 100
            
            if prob > 0.5:
                st.error(f"🚨 *FAKE* ({conf:.2f}%)")
            else:
                st.success(f"✅ *REAL* ({100-conf:.2f}%)")
                
            fig, ax = plt.subplots(figsize=(10, 3))
            librosa.display.specshow(vis, sr=16000, hop_length=160, ax=ax)
            st.pyplot(fig)
    os.remove(path)

In [ ]:
import os
from pyngrok import ngrok
import time

# --- CONFIGURATION ---
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"  # <--- PASTE TOKEN HERE

# 1. Authenticate
ngrok.set_auth_token(NGROK_TOKEN)

# 2. Run Streamlit in the background
# We use nohup to keep it running even if we move to another cell
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

# 3. Open the Tunnel
# Give streamlit a second to start
time.sleep(3)
try:
    # Close any existing tunnels to avoid errors
    ngrok.kill()
    
    # Open new tunnel to port 8501
    public_url = ngrok.connect(8501).public_url
    print(f"🚀 Streamlit is active! Click here: {public_url}")
except Exception as e:
    print(f"Error: {e}")#